In [1]:
import pandas as pd
import torch
import time
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM
)

from tqdm import tqdm

In [7]:
test_df = pd.read_csv('/kaggle/input/refomed/refomed_test.csv', sep='\t')
val_df = pd.read_csv('/kaggle/input/refomed/refomed_val.csv', sep='\t')

plm2key = {"moussaKam/barthez-orangesum-abstract": "barthez-orangesum",
           #"moussaKam/mbarthez-dialogue-summarization":"mbarthez-dialogue"
           #,"moussaKam/mbarthez":"mbarthez"
           #,"moussaKam/barthez":"barthez"
           #,
           #"amasi/barthez-orange-ft": "ftbarthez"
          }

device = 'cpu'

In [8]:
for plm in list(plm2key.keys()):
    
    barthez_tokenizer = AutoTokenizer.from_pretrained(plm)
    barthez_model = AutoModelForSeq2SeqLM.from_pretrained(plm).to(device)
    barthez_model.eval()
    for ip,prompt_template in enumerate(['','',
                "Expliquez-moi, en termes simples ou paraphrase ou définition :",
                "Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :",
              ]):
        if ip <2:
            continue
        print("running", plm,"for p",ip+1)

        for T in [25]:

            queries1 = []
            for k in test_df.term:
                queries1.append(prompt_template+f' {k}')

            queries2 = []
            for k in val_df.term:
                queries2.append(prompt_template+f' {k}')

            responses1 = []
            tt1 = []
            responses2 = []
            tt2 = []
            # ================================ test datframe =====================================
            
            for q in tqdm(queries1):
                t1 = time.time()
                input_ids = torch.tensor(
                    [barthez_tokenizer.encode(q, add_special_tokens=True)]
                )

                predict = barthez_model.generate(input_ids.to(device), max_new_tokens=T)[0]
                answer = barthez_tokenizer.decode(predict, skip_special_tokens=True)
                t2 = time.time()
                responses1.append(answer)
                tt1.append(t2-t1)

            pd.DataFrame({"prompts":queries1, "zsa": responses1, "time_taken":tt1}).to_csv(f'final_FT_{plm2key[plm]}_generation-p{ip}-test-{T}-redo.csv', sep='\t', index=False)
            # ================================ dev datframe =====================================

            for q in tqdm(queries2):
                t1 = time.time()
                input_ids = torch.tensor(
                    [barthez_tokenizer.encode(q, add_special_tokens=True)]
                )

                predict = barthez_model.generate(input_ids.to(device), max_new_tokens=T)[0]
                answer = barthez_tokenizer.decode(predict, skip_special_tokens=True)
                t2 = time.time()
                responses2.append(answer)
                tt2.append(t2-t1)
                #break
            pd.DataFrame({"prompts":queries2, "zsa": responses2, "time_taken":tt2}).to_csv(f'final_{plm2key[plm]}_generation-p{ip}-dev-{T}-redo.csv', sep='\t', index=False)
            

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


running moussaKam/barthez-orangesum-abstract for p 3


100%|██████████| 1063/1063 [09:36<00:00,  1.84it/s]


running moussaKam/barthez-orangesum-abstract for p 4


100%|██████████| 1063/1063 [07:45<00:00,  2.29it/s]
